In [112]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import datetime
from geographiclib.geodesic import Geodesic
plt.rcParams['figure.max_open_warning'] = 0

In [113]:
id_end = pd.read_csv("id_end.csv",parse_dates = [["date","time"]])

In [114]:
#加载训练数据
submit1 = pd.read_csv("sub28_GBRT20_41.csv")
submit1 = submit1[~submit1.id.isin(id_end.id)]

In [115]:
submit0 = pd.DataFrame(columns=['id','date','time','lng_temp','lat_temp'])
from tqdm import tqdm
for i in tqdm(range(id_end.shape[0])):
    new = id_end[i:i+1].copy()
    for j in range(96):
        new["date"] = new.date_time.dt.strftime("%Y/%m/%d")
        new["time"] = new.date_time.dt.strftime("%H:%M")
        new.date_time = new.date_time + datetime.timedelta(minutes=30)
        new["lng_temp"] = np.average([new.lng.values[0],new.lng_end.values[0]],weights=[(96-j),j])
        new["lat_temp"] = np.average([new.lat.values[0],new.lat_end.values[0]],weights=[(96-j),j])
        submit0 = submit0.append(new[['id','date','time','lng_temp','lat_temp']],ignore_index=True)

100%|████████████████████████████████████████████████████████████████████████████████| 113/113 [00:39<00:00,  2.87it/s]


In [116]:
submit0.rename(columns={'lng_temp':'lng', 'lat_temp':'lat'}, inplace = True)

In [117]:
submit_dataset = pd.concat([submit0,submit1],axis=0)

In [120]:
#排序
test_dataset = pd.read_csv("test.csv",parse_dates = [["date","time"]])
sub_order = test_dataset.id.tolist()
submit_dataset = submit_dataset.reset_index()
submit_dataset['id'] = submit_dataset['id'].astype('category')
submit_dataset['id'].cat.set_categories(sub_order, inplace=True) # inplace = True，使 set_categories生效
submit_dataset.sort_values(['id','index'], ascending=[True,True],inplace=True)
submit_dataset = submit_dataset.drop("index",axis = 1)

In [122]:
submit_dataset.to_csv("sub29_GBRT20_41_fix.csv",index=False)

In [121]:
submit_dataset.groupby("id").count()

,date,time,lng,lat
id,,,,
14983,96,96,96,96
22597,96,96,96,96
26303,96,96,96,96
29235,96,96,96,96
30698,96,96,96,96
...,...,...,...,...
974010,96,96,96,96
990797,96,96,96,96
996698,96,96,96,96
